Create by Tianze (Steven) Shou during the RETTL project in Summer 2022 in Vincent's lab under the mentorship of Shamya Karumbaiah. 

This file is serves as a pipeline that takes in Pozyx position data, detect stops according to given parameters (duration and radius), and compare with the distilled observation log event data. 

In [38]:
# python setup 
import pandas as pd
import stop_detection as sd 
import numpy as np
import copy

In [39]:
# function definition 
def isWithinStop(row, stop): 

    """
    :param row: a pandas dataframe with only one row, has to have column named `time_stamp` 
    :param stop: a stop represented by a tuple, denoting the start and end timestamp of a stop
    :return: returns a boolean of whether this row's timestamp is within the given stop 
    """

    assert(type(stop) == tuple and len(stop) == 2) 

    stopStart, stopEnd = stop 
    rowTimestamp = row.loc["time_stamp"]

    return stopStart <= rowTimestamp and rowTimestamp <= stopEnd



def getStopEvent(posDF, stops): 

    """
    This function generates the events and centroids of classroom actor's stops
    :param posDF: a pandas dataframe denoting the position data of an real-world object. Must have columns `chosen_X`, `chosen_Y`, and `time_stamp` 
    :param stops: an array of tuples denoting the start and end timestamps of stops. Usually values returned by sd.getStops()
    :return: returns a tuple of two arrays. The first is an array of strings, denoting the stopping events; the second is a string of tuple points, denoting stop centroid coordinates 
    """

    # values to be returned 
    events = [] 
    centroids = []

    i, j = 0, 0 # i is index for posDF; j is for indexing the list stops
    while(j < len(stops) ): 
        currStop = stops[j] 
        currStopStartInd = i
        inStop = False

        while(i < len(posDF) and isWithinStop(posDF.loc[i], currStop)): 
            inStop = True # indicate to the following code that we do run into the current stop 
            i += 1
            
        if(inStop): # did run into a stop, currStopInd and i shoud not be the same
            assert(currStopStartInd != i)

            rows = posDF.loc[currStopStartInd:i-1] # rows that are within currStop 
            points = sd.cols2tuples(rows.chosen_X, rows.chosen_Y) 
            centroid = sd.getCentroid(points) # get the centroid of current stop 
            assert(type(centroid) == tuple and len(centroid) == 2) # ensures that centroid is a point represented by a tuple

            event = "Stopping in location: " + str(centroid) # format event in string
            
            # need to append multiple events since we are treating stop as a continuous event now
            for k in range(i - currStopStartInd):
                events.append(event) 
                centroids.append(centroid) 

            j += 1 # we have found all rows corresponding to the current stop, go to next stop

        else: # did not run into the current stop 
            assert(currStopStartInd == i)

            # not stopping event, denote as moving 
            events.append("Moving") 
            centroids.append(np.nan)

            # need to go to next row in position dataframe 
            i += 1
    
    # j reaches the end of stop list, but we still need to populate the events list to the same length as the original position dataframe 
    while(i < len(posDF)): 
        events.append("Moving") 
        centroids.append(np.nan)
        i += 1

    assert( len(events) == len(posDF) and len(events) == len(centroids) )
    return events, centroids 


def getClosestObjs(actorDF, objDF, rng): 
    """
    This function returns a list of tuples, where the list is of the same length as actorDF. Tuples contain the names of top objects closest to the centroid stopping points specified in actorDF. Length of tuples are specified by numOfObjs parameter
    
    :param actorDF: pandas dataframe documenting the position of an `actor` by continuous unix timestamp. Must contain column `centroid` 
    :param objDF: pandas dataframe documenting the coordinates of all classroom objects. Must have columns: `object`, `X`, and `Y` 
    :param rng: range parameter; of any classroom objects is with the range distance of the stop centroid, this object gets thrown to the set of objects
    :return: returns a list of tuples. List is of the same length as actorDF. Tuples contain top objects closest to centroids of stops 
    """
    assert(type(rng) == int or type(rng) == float) 
    assert(rng > 0)

    closestObjs = [] # value to be returned, going to contain dictionaries in {<objName1>:<distance1>, <objName2>:<distance2>} format 
    centroids = actorDF.centroid # centroid points for stops, represented by tuples of two ints 
    objPoints = sd.cols2tuples(objDF.X, objDF.Y) # X Y coordinates for classroom objects 
    objNames = objDF.object 
    assert(len(objNames) == len(objPoints)) # these two list/series should have one-to-one corresponding relation 

    i = 0 # indexing for centroids 
    while(i < len(centroids)): 

        centroid = centroids.iloc[i]
        if( np.any(np.isnan(centroid)) ): # actor is not in a stop 
            closestObjs.append(np.nan)

        elif(i - 1 >= 0 and centroids[i-1] == centroid): # if this current centroid is not the first one in the dataframe, and the previous centroid is the same as the current
            objDistDict = copy.deepcopy(closestObjs[len(closestObjs)-1]) # copy the previous object-distance dictionary
            closestObjs.append(objDistDict) # then append the copy

        else: # this means that we need to go through the coordinates of all the classroom objects to find these within range and append them to closetObjs list 
            objDistDict = dict() # create an empty dictionary to hold the entries in the future 
            
            j = 0
            while(j < len(objNames)): 
                if( sd.getDist(centroid, objPoints[j]) < rng ): # if object j within range
                    objDistDict[ objNames[j] ] = sd.getDist(centroid, objPoints[j]) # create a new entry as <object name>:<distance to centroid>
                j += 1

            closestObjs.append(objDistDict) 

        i += 1

    assert(len(closestObjs) == len(actorDF)) # ensure that output length is correct 
    return(closestObjs)

def isEmpty(obj): 
    return not bool(obj)


def getObsInTimeframe(obsDF, timeframeStart, timeframeEnd): 
    obsTimestamps = obsDF["timestamp"] 
    # timestamps of observation data are ensured to be monotonically sorted 
    timeframeStartInd = obsTimestamps.searchsorted(timeframeStart) 
    timeframeEndInd = obsTimestamps.searchsorted(timeframeEnd) 
    return obsDF.loc[timeframeStartInd:timeframeEndInd] 

def calcTriangulationScoreAndPercentages(posDF, obsDF, reward=10, penalty=3, timeframe=10): 

    """
    :param posDF: distilled pozyx position data with stopping event and possible subjects specified 
    :param obsDF: distilled observation log data. See observation_distilled_sprint1_shou.tsv for an example 
    :param reward: reward points given when true subject in observaiton log appears in the set of possible subjects 
    :param penalty: penalty points deducted when incorrent subjects appear in possible subject set 
    :param timeframe: specified how many seconds we look back in time to find the correct subject, unit is second. 
    :return: returns the triangulation validation score for a combination of parameters; higher score means better alignment between modalities 
    """

    assert(reward > 0 and penalty > 0 and timeframe > 0)

    # these two counts are for the calculation of recall 
    rightMatchCount = 0 
    totalMatchCount = 0 
    # these two counts are for the calculation of ~precision 
    rightGuessCount = 0
    totalGuessCount = 0
    # score of kind of the loss function here 
    score = 0

    i = 0  # indexing for observation data 
    while(i < len(obsDF)): 
        obsRow = obsDF.iloc[i] 
        obsEvent = obsRow["event"] # event name specified in observation data 

        # events that we can to valid with position data 
        if(obsEvent == "Talking to student: ON-task" or
           obsEvent == "Talking to student: OFF-task" or 
           obsEvent == "Talking to small group: ON-task" or 
           obsEvent == "Talking to small group: OFF-task"): 

            trueSubjects = obsRow["subject"] # get the true subject(s) from observation data 
            assert(type(trueSubjects) == str and trueSubjects != "") # should now be a string but not empty, in format like "12;13"
            trueSubjects = trueSubjects.split(";") # split by semicolon since seat numbers are demilited by semicolons in distilling process 

            # we look both back and forward in time in position dataframe to check for occurrence of the true subject 
            back = timeframe / 2
            forward = timeframe - timeframe / 2
            assert(back + forward == timeframe)
            timeframeCenter = obsRow["timestamp"] 
            timeframeStart = timeframeCenter - back 
            timeframeEnd = timeframeCenter + forward 
            # filter the position dataframe to get the rows within the timeframe and stopping 
            posInTimeframe = posDF[timeframeStart < posDF["time_stamp"]][posDF["time_stamp"] < timeframeEnd]
            subjSets = posInTimeframe["possibleSubjects"]
            
            for ind in range(len(trueSubjects)): # go thru trueSubjects list to see if they are included in possible subject sets 
                assert( trueSubjects[ind].isdigit() ) # ensure that each subject is a seat number 

                trueSubject = "seat" + trueSubjects[ind] # convert to "seat12" format to align with position data 
                bookMarkedSet = None # we bookmark the set we have seen to avoid counting repeating guesses/matches, only unique sets 

                for subjSet in subjSets: 
                    if(type(subjSet) == dict): # this means that the teacher is detected to be stopping 
                        if(isEmpty(subjSet)): 
                            score -= penalty # penalize if nothing is in the set 
                        elif(trueSubject in subjSet): 
                            score += reward # reward if true subject is in the set 
                            score -= penalty * max(len(subjSet) - 1, 0) # penalize for every wrong guess in the set 
                        else: 
                            score -= penalty * len(subjSet) # penalize since all guesses are wrong 
                    else: # teacher motion is detected, so subject set is NaN 
                        assert(np.isnan(subjSet)) 
                        score -= penalty # penalize as if it is an empty set 

                    # do counting for the percentages only to unique subject sets 
                    # we have seen this before, so skip
                    if(subjSet == bookMarkedSet): 
                        pass 
                    # special case where subjSet is NaN, going to skip 
                    elif(not isinstance(subjSet, dict)): 
                        pass
                    # we have not seen this, bookmark this set and do counting 
                    else: 
                        bookMarkedSet = subjSet
                        totalGuessCount += len(subjSet) # the number of guesses is the number of guessed subjects 
                        totalMatchCount += 1 # one match attempt for each unique set 
                        if(trueSubject in subjSet): 
                            rightMatchCount += 1 # a correct match detected 
                            rightGuessCount += 1 # a correct guess detected 

        i += 1

    # we are going to run through position data again just to see if there is any stopping rows in the position data that does not match to any timeframe. 
    j = 0
    while(j < len(posDF)): 
        posRow = posDF.iloc[j] 
        if(posRow["event"] == "Moving"): 
            j += 1
            continue # only care about stopping rows 

        # get the starting and ending timestamp of the timeframe 
        back = timeframe / 2
        forward = timeframe - timeframe / 2
        assert(back + forward == timeframe)
        timeframeCenter = posRow["time_stamp"] 
        timeframeStart = timeframeCenter - back 
        timeframeEnd = timeframeCenter + forward

        # get all the observation events within the timeframe of the stopping row
        obsInTimeframe = getObsInTimeframe(obsDF, timeframeStart, timeframeEnd) 
        obsEvents = obsInTimeframe["event"] 

        # if any stopping event does exist in the stopping row's timeframe, then we don't care; if not, penalize 
        if("Talking to student: ON-task" in obsEvents or 
           "Talking to student: OFF-task" in obsEvents or
           "Talking to small group: ON-task" in obsEvents or
           "Talking to small group: OFF-task" in obsEvents): 
            pass 
        else: 
            score -= penalty 

        j += 1


    return score, rightMatchCount / totalMatchCount, rightGuessCount / totalGuessCount 


def getPercentages(posStops, obsStops, timeframe, epsilon=0.01): 

    """
    :param posStops: stops datamined from position data, formatted as [(start_stop_1, end_stop_1), (start_stop_2, end_stop_2), ... ] 
    :param obsStops: stops datamined from observation data, formatted as [start_stop_1, start_stop_2, ... ] 
    :param timeframe: timeframe parameter specified in triangulation model 
    :return: returns three percentages, in-both%, in-position-only%, and in-observation-only%
    """

    assert(type(posStops) == list and type(obsStops) == list) 
    posStopsCount = len(posStops) 
    obsStopsCount = len(obsStops) 
    inPosCount = 0 # number of stops only in position data, not in observation data 
    inObsCount = 0 # number of stops only in observation data, not in position data 

    # loop thru position stops to get these only in position not in observation 
    # use the fact that obsStop is a strictly increasing list of ints 
    for posStopStart, posStopEnd in posStops: 

        back = timeframe / 2
        forward = timeframe - back 
        TFstart = posStopStart - back # timeframe start
        TFend = posStopEnd + forward  # timeframe end 

        for obsStop in obsStops: 

            if(obsStop < TFstart): 
                # observation stop is before the timeframe setup by position stop, continue to the next observation stop
                continue 
            elif(TFstart <= obsStop and obsStop <= TFend): 
                # this position stop is corresponding to this observation stop, so break 
                break
            else: 
                # when gets here, it means that this position stop is not corresponding to any observation stops
                assert(TFend < obsStop)
                inPosCount += 1
                break 

    # loop thru observation stops to get these only in observation not in position 
    # use the fact that posStops is a strictly increasing list of int tuples 
    for obsStop in obsStops: 
        for posStopStart, posStopEnd in posStops: 

            back = timeframe / 2
            forward = timeframe - back 
            TFstart = posStopStart - back # timeframe start
            TFend = posStopEnd + forward  # timeframe end 

            if(TFend < obsStop): 
                # observation stop is before the timeframe setup by position stop, continue to the next observation stop
                continue 
            elif(TFstart <= obsStop and obsStop <= TFend): 
                # this position stop is corresponding to this observation stop, so break 
                break
            else: 
                # when gets here, it means that this position stop is not corresponding to any observation stops
                assert(obsStop < TFstart)
                inObsCount += 1
                break 

    # now in-position-only and in-observation-only counts have been calculated 
    # we need to get in-both count 
    inBothCount = int((obsStopsCount + posStopsCount - inObsCount - inPosCount) / 2) 
    assert(inBothCount > 0) # safety check 
    totalCount = inBothCount + inPosCount + inObsCount

    # return the percentages 
    return inBothCount / totalCount, inPosCount / totalCount, inObsCount / totalCount 




In [41]:
# parameter sweep chunk 

# PLEASE LOOK FOR TODO'S AS THEY ARE NECESSARY CHANGES TO SETUP THIS CODE CHUNK 

if __name__ == "__main__": 

    # read teacher position data
    teacherPos = pd.read_csv("teacher_position_sprint1_shou.csv", index_col=False) 
    objPos = pd.read_csv("seating_chart_x_y_seat_only_sprint1_shou.csv", index_col=False) 
    obsLog = pd.read_csv("observation_distilled_sprint1_shou.tsv", sep="\t", index_col=False) 
    outputFileName = "parameter_sweep_master_sprint1_shou.csv" # TODO: CHANGE ME !!!

    # TODO: CHANGE THE THREE PARAMETERS BELOW 
    reward = 2
    penalty = 1
    timeframe = 20 # unit is second 

    # TODO: dataframe to hold the result of the sweep
    sweepDF = pd.DataFrame(columns=["duration", "radius", "range", 
                                    "score", "perc_of_stops_in_both", "perc_of_stops_in_pos", "perc_of_stops_in_obs",
                                    "right_match_percentage", "right_guess_percentage", 
                                    "reward", "penalty", "timeframe"]) 
    #sweepDF = pd.read_csv("parameter_sweep_master_sprint1_shou.csv", index_col=False) 

    for duration in np.arange(3, 30, step=4): # TODO: CHANGE START END AND STEP
        for radius in np.arange(200, 2000, step=200): # TODO: CHANGE START END AND STEP
            
            # call stop detection utility, get starting and end timestamp for each stop with corresponding parameters (duration, radius)
            teacherStops = sd.getStops(teacherPos.chosen_X, teacherPos.chosen_Y, 
                                    teacherPos.time_stamp, teacherPos.periodID, 
                                    teacherPos.dayID, duration, radius) 
            obsStops = sd.getStopsFromObs(obsLog) 

            inBoth, inPos, inObs = getPercentages(teacherStops, obsStops, timeframe) 

            events, centroids = getStopEvent(teacherPos, teacherStops) 
            teacherPos["event"] = events # populate event column for teacher positon dataframe 
            teacherPos["centroid"] = centroids 

            for rng in np.arange(100, 1500, step=200): # TODO: CHANGE START END AND STEP
                rng = int(rng)

                # check if the current set of parameters have been calculated 
                filteredDF = sweepDF[ sweepDF["duration"] == duration ]
                filteredDF = filteredDF[ filteredDF["radius"] == radius ]
                filteredDF = filteredDF[ filteredDF["range"] == rng ]
                filteredDF = filteredDF[ filteredDF["reward"] == reward ]
                filteredDF = filteredDF[ filteredDF["penalty"] == penalty ] 
                filteredDF = filteredDF[ filteredDF["timeframe"] == timeframe ] 
                # if yes, skip this set and go to the next 
                if( len(filteredDF) >= 1 ): 
                    print("Parameter set", duration, radius, rng, "skipped")
                    print("###########################################")
                    continue 
                

                # get the objects and their correpsonding distances to centroid within rng (range) 
                subjects = getClosestObjs(teacherPos, objPos, rng)
                teacherPos["possibleSubjects"] = subjects 

                score, rightMatchPercentage, rightGuessPercentage = calcTriangulationScoreAndPercentages(teacherPos, obsLog, reward=reward, penalty=penalty, timeframe=timeframe) 

                # update sweep dataframe with each parameter set we test for 
                newRow = {"duration": [duration], 
                        "radius": [radius], 
                        "range": [rng], 
                        "score": [score], 
                        "perc_of_stops_in_both": inBoth, 
                        "perc_of_stops_in_pos": inPos, 
                        "perc_of_stops_in_obs": inObs, 
                        "right_match_percentage": [rightMatchPercentage], 
                        "right_guess_percentage": [rightGuessPercentage],
                        "reward": [reward], 
                        "penalty": [penalty], 
                        "timeframe": [timeframe]} 
                newDF = pd.DataFrame(newRow)
                sweepDF = pd.concat([sweepDF, newDF], ignore_index=True)

                print("Parameter set (duration, radius, range) is:", duration, radius, rng) 
                print("Score is:", score)
                print("Three percentages are (in both, in position only, in observation only):", inBoth, inPos, inObs) 
                print("Right match percentage is", rightMatchPercentage) 
                print("Right guess percentage is", rightGuessPercentage) 
                print("###########################################")

            # TODO: do you want to output your data to a csv? 
            sweepDF.to_csv(outputFileName, index=False)
            print("Changes to the data file has been saved to", outputFileName)
            print("###########################################")

/var/folders/jk/t2z3d3zd447g4wlv0rvml1400000gn/T/ipykernel_23622/1762649444.py:180: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  posInTimeframe = posDF[timeframeStart < posDF["time_stamp"]][posDF["time_stamp"] < timeframeEnd]


Parameter set (duration, radius, range) is: 3 200 100
Score is: -22971
Three percentages are (in both, in position only, in observation only): 0.3582887700534759 0.6250742721330956 0.016636957813428402
Right match percentage is 0.006109979633401222
Right guess percentage is 0.375
###########################################
Parameter set (duration, radius, range) is: 3 200 300
Score is: -22506
Three percentages are (in both, in position only, in observation only): 0.3582887700534759 0.6250742721330956 0.016636957813428402
Right match percentage is 0.05702647657841141
Right guess percentage is 0.3146067415730337
###########################################
Parameter set (duration, radius, range) is: 3 200 500
Score is: -20532
Three percentages are (in both, in position only, in observation only): 0.3582887700534759 0.6250742721330956 0.016636957813428402
Right match percentage is 0.26272912423625255
Right guess percentage is 0.32575757575757575
###########################################


KeyboardInterrupt: 

In [96]:
# testing chunk 
if __name__ == "__main__": 
    # read teacher position data
    teacherPos = pd.read_csv("teacher_position_sprint1_shou.csv", index_col=False) 
    objPos = pd.read_csv("seating_chart_x_y_sprint1_shou.csv", index_col=False) 
    obsLog = pd.read_csv("observation_distilled_sprint1_shou.tsv", sep="\t", index_col=False) 

    # parameter setting 
    duration = 15
    radius = 500
    rng = 2000

    # call stop detection utility, get starting and end timestamp for each stop 
    teacherStops = sd.getStops(teacherPos.chosen_X, teacherPos.chosen_Y, 
                            teacherPos.time_stamp, teacherPos.periodID, 
                            teacherPos.dayID, duration, radius) 

    events, centroids = getStopEvent(teacherPos, teacherStops) 
    teacherPos["event"] = events # populate event column for teacher positon dataframe 
    teacherPos["centroid"] = centroids 

    # get the objects and their correpsonding distances to centroid within rng (range) 
    subjects = getClosestObjs(teacherPos, objPos, rng)
    teacherPos["possibleSubjects"] = subjects 
